In [1]:
import pandas as pd

pre_separated_data = True

## EDA

**Dados de treino (2022)**

Este é o conjunto de dados que você e sua equipe irão usar para trabalhar, criar o modelo, fazer testes e desenvolver a solução final. 

Transações: Data, PDV, Produto, Quantidade, Faturamento. 

Cadastro de produtos: Produto, Categoria, Descrição, + até 4 atributos.

Cadastro de PDVs: PDV, On/Off Prem, Categoria (c-store, g-store, liquor etc.), Zipcode. 

In [9]:
transacoes_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
cadastro_prod_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet"
cadastro_pdv_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"

In [ ]:
if not pre_separated_data:
    import polars as pl

    df3 = pl.read_parquet(cadastro_prod_path)
    print(df3.shape)

    total_rows = df3.shape[0]
    chunk_size = total_rows // 10  # Integer division to get the chunk size

    for i in range(10):
        start_index = i * chunk_size
        end_index = start_index + chunk_size

        if i == 9:
            chunk = df3.slice(start_index)
        else:
            chunk = df3.slice(start_index, chunk_size)

        output_filename = (
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )

        chunk.write_parquet(output_filename)
        print(f"Written {output_filename}")

In [ ]:
df_transacoes = pd.read_parquet(transacoes_path)
df_cadastro_pdv = pd.read_parquet(cadastro_pdv_path)

if pre_separated_data:
    all_dfs = []
    for i in range(10):
        df3 = pd.read_parquet(
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )
        all_dfs.append(df3)
    df_cadastro_prod = pd.concat(all_dfs, ignore_index=True)
    del all_dfs

In [11]:
df_transacoes.shape, df_cadastro_pdv.shape, df_cadastro_prod.shape

((6560698, 11), (14419, 4), (192356316, 8))

In [12]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


`internal_store_id` de `df_transacoes` corresponde ao `pdv` da tabela `df_cadastro_pdv`. \
`internal_product_id` de `df_transacoes` corresponde ao `produto` da tabela `df_cadastro_prod`.

In [13]:
df_cadastro_pdv.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [14]:
df_cadastro_prod.head()

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery


Arquivo de previsão no formato CSV ou Parquet, com as seguintes colunas:

| semana | pdv  | produto | quantidade |
|--------|------|---------|------------|
| 1      | 1023 | 123     | 120        |
| 2      | 1045 | 234     | 85         |
| 3      | 1023 | 456     | 110       |

No caso do csv, utilize “;” como caractere separador (exemplo: 1;1023;123;120) e encoding UTF-8.

**semana (número inteiro)**: número da semana (1 a 4 de janeiro/2023) \
**pdv (número inteiro)**: código do ponto de venda \
**produto (número inteiro)**: código do SKU \
**quantidade (número inteiro)**: previsão de vendas


## Organizar dados para obter Y

In [ ]:
df_transacoes["semana"] = (
    (pd.to_datetime(df_transacoes["transaction_date"]).dt.dayofyear - 1) // 7
) + 1

In [33]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,semana
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,28
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,12
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,36
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,37
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,7


In [34]:
df_transacoes.value_counts(["semana", "transaction_date"]).sort_index().head(30)

semana  transaction_date
1       2022-01-01            170
        2022-01-02           1112
        2022-01-03           8833
        2022-01-04          20934
        2022-01-05          18039
        2022-01-06          17456
        2022-01-07          12957
2       2022-01-08            319
        2022-01-09           1601
        2022-01-10          22159
        2022-01-11          21377
        2022-01-12          19034
        2022-01-13          14901
        2022-01-14          11963
3       2022-01-15            373
        2022-01-16            732
        2022-01-17           8744
        2022-01-18          23937
        2022-01-19          21639
        2022-01-20          18973
        2022-01-21          14709
4       2022-01-22            589
        2022-01-23           2348
        2022-01-24          25546
        2022-01-25          21369
        2022-01-26          19286
        2022-01-27          16072
        2022-01-28          12957
5       2022-01-29     

In [ ]:
df_transacoes_grouped = (
    df_transacoes.groupby(["semana", "internal_store_id", "internal_product_id"])
    .agg({"quantity": "sum"})
    .reset_index()
)
df_transacoes_grouped = df_transacoes_grouped.rename(
    columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade",
    }
)

In [39]:
df_transacoes_grouped.sort_index().head(20)

,semana,pdv,produto,quantidade
0,1,1001371918471115422,1029370090212151375,2.0
1,1,1001371918471115422,1120490062981954254,18.0
2,1,1001371918471115422,2239307647969388381,2.0
3,1,1001371918471115422,4353552881410365573,7.0
4,1,1001371918471115422,4797439216678436447,1.0
5,1,1001371918471115422,5913249594472983850,7.0
6,1,1001371918471115422,7852170368684115755,15.0
7,1,1001371918471115422,78720985238870578,2.0
8,1,1001371918471115422,8222093130128661995,2.0
9,1,1001371918471115422,8305237228324117368,2.0
